In [6]:
using System.Collections.Immutable;

public abstract record Tree<T>(T Value)
{
    public record Node(T Value, ImmutableList<Tree<T>> Children) : Tree<T>(Value);
    public record Leaf(T Value) : Tree<T>(Value);
}

new Tree<int>.Node(
    4,
    ImmutableList.Create<Tree<int>>(
        new Tree<int>.Leaf(4),
        new Tree<int>.Leaf(3)
))

Node { Value = 4, Children = System.Collections.Immutable.ImmutableList`1[Submission#7+Tree`1[System.Int32]] } Children index value 0 Leaf { Value = 4 } Value 4 1 Leaf { Value = 3 } Value 3 Value 4

In [7]:
static TAcc AggregateBack<TAcc, T>(this ImmutableList<T> list, TAcc seed, Func<TAcc, T, TAcc> func)
{
    return list.Aggregate<T, Func<TAcc, TAcc>>(
        acc => acc,
        // foldBack
        (acc, x) => (TAcc nextAcc) => acc(func(nextAcc, x))
        // fold
        //(acc, x) => (TAcc nextAcc) => func(acc(nextAcc), x)
    ).Invoke(seed);
}

In [8]:
// https://stackoverflow.com/questions/75323058/a-foldback-function-for-tree-in-f

static TAcc AggregateBack<TAcc, T>(this Tree<T> tree, TAcc seed, Func<TAcc, T, TAcc> folder)
{
    Func<TAcc, TAcc> Aggregate(Func<TAcc, TAcc> acc, Tree<T> tree)
    {
        return ((Func<Func<TAcc, TAcc>>)(tree switch
        {
            Tree<T>.Leaf leaf => () =>
            {
                Func<TAcc, TAcc> acc1 = (inner) => folder(acc(inner), leaf.Value);
                return acc1;
            },
            Tree<T>.Node node => () =>
            {
                Func<TAcc, TAcc> acc1 =
                    (inner) => node.Children.AggregateBack(inner, (_acc, tree) => Aggregate(acc, tree).Invoke(_acc))
                ;
                Func<TAcc, TAcc> acc2 = (inner) => folder(acc1(inner), node.Value);
                
                return acc2;
            }
        }))();
    }

    return Aggregate(a => a, tree).Invoke(seed);
}

var treeSample =
    new Tree<string>.Node("node1", ImmutableList.Create<Tree<string>>(
            new Tree<string>.Leaf("subnode1"),
            new Tree<string>.Node("node1.1", ImmutableList.Create<Tree<string>>(
                new Tree<string>.Leaf("subnode1.1"),
                new Tree<string>.Node("node1.2", ImmutableList.Create<Tree<string>>(
                    new Tree<string>.Leaf("leaf1.2")
                ))
            ))
    ));

treeSample.AggregateBack(">>seed<<", (value, acc) => $"{value} -> {acc}")


>>seed<< -> leaf1.2 -> node1.2 -> subnode1.1 -> node1.1 -> subnode1 -> node1